In [1]:
import pandas as pd
from requests_html import HTMLSession
import warnings

warnings.filterwarnings('ignore')

In [2]:
url = 'https://www.basketball-reference.com/teams/'

teams = pd.read_html(url, attrs = {'id': 'teams_active'})[0]
teams = teams[teams['To'].str[-2:] == '21']  
teams = teams.drop_duplicates(subset = ['Franchise']).reset_index(drop = True)

# Add team abbreviations
abbrev_url = 'https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations'
abbrev = pd.read_html(abbrev_url, header = 0)[0]
teams = pd.concat([teams, abbrev['Abbreviation/Acronym']], axis = 1)

teams.index.rename('team_id', inplace = True)
teams[['City', 'Name']] = teams['Franchise'].str.rsplit(' ', 1, expand = True)
teams = teams[['City', 'Name', 'Abbreviation/Acronym', 'From', 'To', 'Yrs', 'G', 'W', 'L', 'W/L%', 'Plyfs', 'Div', 'Conf', 'Champ']]

teams.head()

,City,Name,Abbreviation/Acronym,From,To,Yrs,G,W,L,W/L%,Plyfs,Div,Conf,Champ
team_id,,,,,,,,,,,,,,
0,Atlanta,Hawks,ATL,1949-50,2020-21,72,5691,2807,2884,0.493,47,11,0,1
1,Boston,Celtics,BKN,1946-47,2020-21,75,5868,3462,2406,0.590,57,31,9,17
2,Brooklyn,Nets,BOS,1967-68,2020-21,54,4366,1907,2459,0.437,29,5,2,2
3,Charlotte,Hornets,CHA,1988-89,2020-21,31,2467,1083,1384,0.439,10,0,0,0
4,Chicago,Bulls,CHI,1966-67,2020-21,55,4434,2258,2176,0.509,35,9,6,6


In [3]:
from collections import defaultdict

teams_dict = defaultdict(dict)

for team_name in teams['Name']:
    current_team = teams[teams['Name'] == team_name]
    
    teams_dict[team_name] = {}
    teams_dict[team_name]['id'] = int(current_team.index.values)
    teams_dict[team_name]['city'] = current_team.City
    teams_dict[team_name]['initials'] = current_team['Abbreviation/Acronym']
    
teams_dict = dict(teams_dict)

In [4]:
def add_team_id(team_name):
    return teams_dict[team_name]['id']

def get_current_standings():
    url = 'https://www.basketball-reference.com/leagues/NBA_2021.html'
    east = pd.read_html(url, attrs= {'id': 'confs_standings_E'})[0]
    west = pd.read_html(url, attrs= {'id': 'confs_standings_W'})[0]
    
    east[['City', 'Name']] = east['Eastern Conference'].str.replace('[^a-zA-Z ]', '').str.rsplit(' ', 1, expand = True)
    west[['City', 'Name']] = west['Western Conference'].str.replace('[^a-zA-Z ]', '').str.rsplit(' ', 1, expand = True)
    east.loc[(east.City == 'Philadelphia'), 'Name'] = '76ers'
    
    west['team_id'] = west['Name'].apply(add_team_id)
    east['team_id'] = east['Name'].apply(add_team_id)

    east = east[['team_id', 'City', 'Name', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']]
    west = west[['team_id', 'City', 'Name', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']]    
    
    return east, west

In [5]:
east_standings, west_standings = get_current_standings()
east_standings.head()

,team_id,City,Name,W,L,W/L%,GB,PS/G,PA/G,SRS
0,22,Philadelphia,76ers,49,23,0.681,—,113.6,108.1,5.28
1,2,Brooklyn,Nets,48,24,0.667,1.0,118.6,114.1,4.24
2,16,Milwaukee,Bucks,46,26,0.639,3.0,120.1,114.2,5.57
3,19,New York,Knicks,41,31,0.569,8.0,107.0,104.7,2.13
4,0,Atlanta,Hawks,41,31,0.569,8.0,113.7,111.4,2.14
